
# Module 4 Session 2 - Quantifying Health Disparities

## Authors: Shivam Sharma & Vincent Lam 

#### Date: May 29, 2024


## Section 1 - Data exploration

## Section 2 - Create case-control cohorts

## Section 3 - Calculate age & sex adjusted prevalences

## Section 4 - Visualize prevalences

## Section 5